In [1]:
import pandas as pd
import numpy as np
import json
from datetime import date
import urllib


In [56]:
# загрузить файл с диска (на работе) и сделать ДФ
js = json.load(open('../../head-to-head-elo.json')).get('data')
df = pd.DataFrame(js)

In [62]:
df

,againstHero,games,hero,losses,shift,wins
0,17,10,81,0,31.104140,10
1,111,10,11,1,28.204101,9
2,98,10,81,1,27.563220,9
3,104,13,42,1,26.226689,12
4,98,14,61,1,25.605029,13
5,28,11,22,2,25.162021,9
6,112,21,102,4,24.842743,17
7,86,10,59,3,23.513774,7
8,75,20,77,3,23.506009,17
9,84,10,82,2,23.479558,8


# Создать файлы с данными по elo для пар героев ко дню проведения матча с мнимальным количеством игр=10

In [ ]:
columns = ['hero', 'kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
           'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
# создать файлы с elo пар героев по каждой дате (с 2017 года используется только патч 7,00+)
year = 2016
for month in range (12, 13):
    for day in range (11, 32):
         # создать правильную ссылку для поиска по всем героям 
        url_heroes =('http://www.datdota.com/heroes/head-to-head-elo?tier=1&valve-event=does-not-matter&threshold=10' +
                     '&patch=7.09&patch=7.08&patch=7.07&patch=7.06&patch=7.05&patch=7.04&patch=7.03&patch=7.02' +
                     '&patch=7.01&patch=7.00&patch=6.88&winner=either&after=01%2F01%2F2011' +
                     '&before={}%2F{}%2F{}&duration=0%3B200&'.format(day, month, year) +
                     'duration-value-from=0&duration-value-to=200') 
        # выгрзить json с предыдущей ссылки
        sauce = urllib.request.Request(url_heroes, headers={'User-agent': 'Mozilla/5.0'} )
        dat = json.loads(urllib.request.urlopen(sauce).read().decode()).get('data')
        df = pd.DataFrame(dat)
        
        # запись фала на диск
        df.to_csv('../tabel with heroes DATDOTA (elo для пар героев)/{}-{}-{}.csv'.format(year, month, day))

# Обработка и создание полноценного ДФ с id матчем и суммарным elo для каждой команды

In [83]:
# загрузить главный ДФ
main = pd.read_csv('../tabel/MAIN TABLE PREMIUM.csv', index_col=0)
main = main[:2]
main

,match_id,start_time,radiant_team_id,radiant_score,dire_team_id,dire_score,radiant_win,radiant_H1,radiant_H2,radiant_H3,...,radiant_P2,radiant_P3,radiant_P4,radiant_P5,dire_P1,dire_P2,dire_P3,dire_P4,dire_P5,league_name
0,2897064589,1483677730,1883502.0,23,350190.0,12,True,86.0,8.0,76.0,...,132851371.0,106573901.0,92423451.0,106809101.0,89871557.0,181716137.0,112377459.0,121052479.0,93616251.0,ESL One Genting 2017
1,2897074199,1483678249,3214108.0,8,1375614.0,37,False,21.0,55.0,71.0,...,86726887.0,87196890.0,40547474.0,47434686.0,139876032.0,117281554.0,82327674.0,149486894.0,87012746.0,ESL One Genting 2017


In [77]:
def elo_heroes_vs_enemies(index, columns_heroes, columns_enemies, main_df):
    # две переменные для записи суммарного elo каждой команды
    radiant_elo = 0
    dire_elo = 0
    for her in columns_heroes:
        # вытащить id героя
        id_heroe = main[her][index]

        # суммировать elo героя против героев противника
        for her_enemy in columns_enemies:
            id_heroe_enemy = main[her_enemy][index]
            try:
                elo =  df['shift'][df['hero'] == id_heroe][df['againstHero'] == id_heroe_enemy].item()
            except:
                elo=0
            radiant_elo += elo
        # записать в основной ДФ elo по каждому герою
        main_df.loc[index, her + 'elo_vs_enemies'] = radiant_elo

In [80]:
# columns = ['kills', 'deaths', 'assists', 'kda', 'avgKal', 'gpm', 'xpm', 'lastHits', 'denies', 'level',
#            'heroDamage', 'towerDamage', 'heroHealing', 'goldSpent']
# создать основной массив, где будут записаны суммарное elo каждого егроя относительно всех героев противника
df_head_to_head_elo_heroes = pd.DataFrame()#main['match_id']

# создать список названий колонок  героев radiant
all_her_rad = main.loc[:,'radiant_H1':'radiant_H5'].columns
# создать список названий колонок  героев radiant
all_her_dir = main.loc[:,'dire_H1':'dire_H5'].columns

for index in main.index:
    
    elo_heroes_vs_enemies(index, all_her_rad, all_her_dir, df_head_to_head_elo_heroes )
    elo_heroes_vs_enemies(index, all_her_dir, all_her_rad, df_head_to_head_elo_heroes )
#     # две переменные для записи суммарного elo каждой команды
#     radiant_elo = 0
#     dire_elo = 0
#     for her in all_her_rad:
#         # вытащить id героя
#         id_heroe = main[her][index]

#         # суммировать elo героя против героев противника
#         for her_enemy in all_her_dir:
#             id_heroe_enemy = main[her_enemy][index]
#             try:
#                 elo =  df['shift'][df['hero'] == id_heroe][df['againstHero'] == id_heroe_enemy].item()
#             except:
#                 elo=0
#             radiant_elo += elo
#         # записать в основной ДФ elo по каждому герою
#         df_head_to_head_elo_heroes.loc[index, her + 'elo_vs_enemies'] = radiant_elo

        

        
#             # создать массив с данными 
#             array = df_data_tabel_for_heroes[df_data_tabel_for_heroes['hero'] == id_heroe].drop('hero', axis=1).values
#             # создать название колонок для определнного героя
#             col = [her + '_' + c for c  in columns]
#             # ДФ для героя по матчу
#             df_heroe = pd.DataFrame(array, columns=col)
#             df_heroe_match = pd.merge(df_heroe_match, df_heroe, 
#                                                left_index=True, right_index=True, how='outer')
#     df_basick_peromances_heroes = pd.concat([df_basick_peromances_heroes, df_heroe_match])
    
#     # ДФ для соединения всех герове матча в один ДФ 
#     df_heroe_match = pd.DataFrame()
#     # дата матча
#     date_match = date.fromtimestamp(main['start_time'][index])
#     # загрузить файл с kda рейтингом героев по дате матча
#     df_data_tabel_for_heroes = pd.read_csv('../tabel with heroes DATDOTA (kda)/{}-{}-{}.csv'.
#                                                format(date_match.year, date_match.month, date_match.day), index_col=0) 

# df_basick_peromances_heroes = df_basick_peromances_heroes.reset_index().drop('index', axis=1)
# df_basick_peromances_heroes['match_id'] = main['match_id']

In [81]:
df_head_to_head_elo_heroes

,radiant_H1elo_vs_enemies,radiant_H2elo_vs_enemies,radiant_H3elo_vs_enemies,radiant_H4elo_vs_enemies,radiant_H5elo_vs_enemies,dire_H1elo_vs_enemies,dire_H2elo_vs_enemies,dire_H3elo_vs_enemies,dire_H4elo_vs_enemies,dire_H5elo_vs_enemies
0,-10.412864,-19.676853,-7.902946,-5.812015,-16.328774,-21.672873,-20.547259,-8.893307,-3.928382,7.767046
1,0.555364,-9.451070,-21.965169,-34.432935,-36.878635,10.995322,23.442587,19.049655,20.862582,32.115204


In [66]:
id_heroe_enemy

9.0